In [1]:
#导入 selenium 第三方库
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
import os
import glob
import re
from pathlib import Path 

DOWNLOAD_DIR = os.path.abspath("jcad")  # 相对路径
chrome_options = Options()
prefs = {
    # 设置默认下载路径
    "download.default_directory": DOWNLOAD_DIR,
    # 禁用下载前的文件保存确认弹窗
    "download.prompt_for_download": False,
    # 关闭PDF文件的内置预览（直接下载而非在浏览器打开）
    "plugins.always_open_pdf_externally": True,
    # 禁用下载通知栏提示
    "download.directory_upgrade": True,
    #自动允许网站下载多个文件（禁用多文件下载确认弹窗）
    "profile.default_content_setting_values.automatic_downloads": 1 
}
chrome_options.add_experimental_option("prefs", prefs)

In [2]:
#指定驱动路径
service = Service(r"./chromedriver.exe")        
driver = webdriver.Chrome(service=service,  options=chrome_options)

In [3]:
# -------------------------- 工具函数：等待下载完成 + 重命名 --------------------------
def wait_for_download_complete(download_dir, timeout=60):
    """
    等待本次下载完成，返回最新下载的文件路径（仅识别本次新增的文件）
    :param download_dir: 下载目录（绝对路径，建议用r前缀）
    :param timeout: 超时时间（秒）
    :return: 本次下载的文件路径
    """
    # 步骤1：记录「下载开始前」的基准文件列表（区分旧文件）
    download_dir = Path(download_dir)  # 用Path简化路径操作
    # 基准列表：下载前已存在的非临时文件（排除所有下载临时后缀）
    temp_suffixes = (".crdownload", ".part", ".tmp")  # 覆盖主流浏览器临时文件
    baseline_files = set()
    for file in download_dir.iterdir():
        if file.is_file() and not file.name.lower().endswith(temp_suffixes):
            baseline_files.add(file.name)

    start_time = time.time()
    latest_new_file = None  # 存储本次下载的新文件
    
    while time.time() - start_time < timeout:
        # 步骤2：遍历目录，筛选「本次新增的文件」（不在基准列表+非临时文件）
        current_files = []
        for file in download_dir.iterdir():
            # 排除临时文件、文件夹、旧文件
            if (file.is_file() 
                and not file.name.lower().endswith(temp_suffixes)
                and file.name not in baseline_files):
                current_files.append(file)
        
        if current_files:
            # 取本次新增文件中「最新修改」的（确保是刚下载完成的）
            latest_new_file = max(current_files, key=lambda x: x.stat().st_mtime)
            # 额外保险：等待文件大小稳定（避免文件还在写入）
            prev_size = -1
            stable_count = 0
            while stable_count < 2:  # 连续2次检查大小不变
                current_size = latest_new_file.stat().st_size
                if current_size == prev_size:
                    stable_count += 1
                else:
                    prev_size = current_size
                    stable_count = 0
                time.sleep(0.5)
            return str(latest_new_file)  # 返回绝对路径
        
        # 步骤3：如果还没新文件，等待1秒再检查（不着急返回）
        time.sleep(1)
    
    # 当前页面刷新, 再尝试
    driver.refresh()
    return None  # 超时未找到新文件


In [4]:
def rename_downloaded_file(old_file_path, new_file_path):
    """
    重命名下载的文件（处理同名文件覆盖）
    :param old_file_path: 原文件路径
    :param new_file_path: 新文件路径
    """
    # 如果新文件已存在，先删除（可选：也可以加后缀避免覆盖）
    if os.path.exists(new_file_path):
        os.remove(new_file_path)
    # 重命名
    os.rename(old_file_path, new_file_path)
    print(f"文件已重命名：{new_file_path}")
    


In [5]:
def is_file_downloaded(download_dir, target_filename, timeout=60):
    """
    判断文件是否下载完成
    """
    start_time = time.time()
    temp_suffixes = [".crdownload", ".part", ".tmp"]  # 浏览器临时文件后缀
    
    while time.time() - start_time < timeout:
        # 遍历下载目录下的所有文件
        for filename in os.listdir(download_dir):
            # 情况1：找到最终文件（无临时后缀）
            if filename == target_filename:
                time.sleep(1)  # 保险：等待文件写入完成
                return True
            # 情况2：找到临时文件（还在下载中）
            elif filename.startswith(target_filename) and any(filename.endswith(suf) for suf in temp_suffixes):
                time.sleep(0.5)  # 下载中，继续等待
                break
        else:
            # 既没找到最终文件，也没找到临时文件 → 可能还没开始下载
            time.sleep(0.5)
    # 超时
    return False

In [6]:
def wait_for_download_complete(download_dir, timeout=60):
    """
    等待本次下载完成，返回最新下载的文件路径（仅识别本次新增的文件）
    :param download_dir: 下载目录（绝对路径，建议用r前缀）
    :param timeout: 超时时间（秒）
    :return: 本次下载的文件路径
    """
    # 步骤1：记录「下载开始前」的基准文件列表（区分旧文件）
    download_dir = Path(download_dir)  # 用Path简化路径操作
    # 基准列表：下载前已存在的非临时文件（排除所有下载临时后缀）
    temp_suffixes = (".crdownload", ".part", ".tmp")  # 覆盖主流浏览器临时文件
    baseline_files = set()
    for file in download_dir.iterdir():
        if file.is_file() and not file.name.lower().endswith(temp_suffixes):
            baseline_files.add(file.name)

    start_time = time.time()
    latest_new_file = None  # 存储本次下载的新文件
    
    while time.time() - start_time < timeout:
        # 步骤2：遍历目录，筛选「本次新增的文件」（不在基准列表+非临时文件）
        current_files = []
        for file in download_dir.iterdir():
            # 排除临时文件、文件夹、旧文件
            if (file.is_file() 
                and not file.name.lower().endswith(temp_suffixes)
                and file.name not in baseline_files):
                current_files.append(file)
        
        if current_files:
            # 取本次新增文件中「最新修改」的（确保是刚下载完成的）
            latest_new_file = max(current_files, key=lambda x: x.stat().st_mtime)
            # 额外保险：等待文件大小稳定（避免文件还在写入）
            prev_size = -1
            stable_count = 0
            while stable_count < 2:  # 连续2次检查大小不变
                current_size = latest_new_file.stat().st_size
                if current_size == prev_size:
                    stable_count += 1
                else:
                    prev_size = current_size
                    stable_count = 0
                time.sleep(0.5)
            return str(latest_new_file)  # 返回绝对路径
        
        # 步骤3：如果还没新文件，等待1秒再检查（不着急返回）
        time.sleep(1)
    
    # 超时抛出异常
    raise TimeoutError(
        f"下载超时！{timeout}秒内未找到本次下载的新文件\n"
        f"下载目录已有旧文件：{list(baseline_files)[:5]}（最多显示5个）"
    )

In [24]:
import time
import os
import re
import platform
import requests
from pathlib import Path
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# -------------------------- 核心配置 --------------------------
DOWNLOAD_ROOT_DIR = os.path.abspath("jcad") 
BATCH_SIZE = 47
START_YEAR = 2025       
START_VOLUME = 9
MIN_YEAR = 2025
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "zh-CN,zh;q=0.9,en;q=0.8",
    "Referer": "https://www.jcad.cn/",
    "Upgrade-Insecure-Requests": "1",
    "Connection": "keep-alive"
}

# -------------------------- 工具函数 --------------------------
def clear_console():
    if platform.system() == "Windows":
        os.system("cls")
    else:
        os.system("clear")

def create_session():
    session = requests.Session()
    retry_strategy = Retry(
        total=3,
        backoff_factor=1,
        status_forcelist=[429, 500, 502, 503, 504]
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount("https://", adapter)
    session.mount("http://", adapter)
    session.headers.update(HEADERS)
    return session

def sanitize_filename(filename, max_length=100):
    if not filename or filename.strip() == "":
        return "unknown_file"
    illegal_chars = r'[\\/:*?"<>|·¥$@#^&(){}[\]~`!%+=,]'
    safe_name = re.sub(illegal_chars, "_", filename.strip())
    safe_name = re.sub(r'\s+', "_", safe_name)
    safe_name = safe_name.strip("_")
    return safe_name if safe_name else "unknown_file"

def extract_pdf_download_urls(page_url, session):
    try:
        response = session.get(page_url, timeout=15)
        response.raise_for_status()
        response.encoding = response.apparent_encoding or "utf-8"
        soup = BeautifulSoup(response.text, "html.parser")
        pdf_links = []
        existing_filenames = set()

        # 找到所有PDF下载按钮
        download_buttons = soup.find_all(
            "a", 
            string="PDF下载",
            onclick=lambda x: x and "downloadpdf" in x
        )
        print(f"\n📌 共找到 {len(download_buttons)} 个PDF下载按钮")

        for btn_idx, btn in enumerate(download_buttons, 1):
            print(f"\n===== 处理第 {btn_idx} 个PDF按钮 =====")
            onclick = btn.get("onclick", "")
            uuid_match = re.findall(r'downloadpdf\(["\']([0-9a-f-]+)["\']', onclick)
            if not uuid_match:
                print("❌ 未提取到UUID，跳过")
                continue
            pdf_uuid = uuid_match[0]
            pdf_url = f"https://www.jcad.cn/article/exportPdf?id={pdf_uuid}"
            print(f"✅ 提取到UUID：{pdf_uuid}")

            # -------------------------- 核心修改：强制匹配目标标题容器 --------------------------
            article_title = ""
            current_elem = btn
            # 向上遍历最多10层父元素
            for _ in range(10):
                current_elem = current_elem.parent
                if not current_elem:
                    break
                # 🔴 只从 article-list-title 容器提取标题（精准锁定目标）
                title_div = current_elem.find("div", class_=re.compile(r"^article-list-title"))
                if title_div:
                    title_a = title_div.find("a")
                    if title_a:
                        article_title = title_a.get_text(strip=True)
                        print(f"✅ 在第 {_+1} 层父元素找到目标标题：【{article_title}】")
                        break
            # -----------------------------------------------------------------------------------

            # 兜底逻辑
            if not article_title:
                print("❌ 未找到目标标题，使用UUID兜底")
                article_title = f"article_{pdf_uuid[:8]}"

            # 文件名处理（去重+清理）
            base_filename = sanitize_filename(article_title)
            final_filename = base_filename
            counter = 1
            while final_filename in existing_filenames:
                final_filename = f"{base_filename}_{counter}"
                counter += 1
            existing_filenames.add(final_filename)
            file_name = f"{final_filename}.pdf"
            print(f"📄 最终文件名：【{file_name}】")

            pdf_links.append((file_name, pdf_url))

        return pdf_links
    except Exception as e:
        print(f"❌ 解析页面失败：{str(e)}")
        import traceback
        traceback.print_exc()
        return []

# -------------------------- 下载函数 --------------------------
def download_pdf_batch(save_dir, pdf_batch, session):
    success_files = []
    for idx, (file_name, pdf_url) in enumerate(pdf_batch, 1):
        save_path = os.path.join(save_dir, file_name)
        if os.path.exists(save_path) and os.path.getsize(save_path) > 100:
            print(f"✅ 已存在，跳过：{file_name}")
            success_files.append(save_path)
            continue
        try:
            print(f"📥 正在下载第{idx}个：{file_name}")
            response = session.get(pdf_url, stream=True, timeout=30)
            response.raise_for_status()
            content_type = response.headers.get("Content-Type", "").lower()
            valid_content_types = ["pdf", "octet-stream", "x-pdf", "binary/octet-stream"]
            if not any(ctype in content_type for ctype in valid_content_types):
                print(f"⚠️ 非标准Content-Type：{content_type}，继续下载...")
            with open(save_path, "wb") as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
            # 验证PDF有效性
            file_size = os.path.getsize(save_path)
            is_valid = False
            if file_size > 100:
                with open(save_path, "rb") as f:
                    if f.read(4) == b"%PDF":
                        is_valid = True
            if is_valid:
                print(f"✅ 下载成功：{file_name}（{file_size/1024:.1f}KB）")
                success_files.append(save_path)
            else:
                os.remove(save_path)
                print(f"❌ 无效PDF文件，已删除")
        except Exception as e:
            print(f"❌ 下载失败：{file_name} - {str(e)}")
            if os.path.exists(save_path):
                os.remove(save_path)
        finally:
            time.sleep(0.5)
    return success_files

# -------------------------- 主逻辑 --------------------------
def main():
    session = create_session()
    year = START_YEAR
    volume = START_VOLUME
    while True:
        try:
            folder_name = f"{year}{volume:02d}"
            save_dir = os.path.join(DOWNLOAD_ROOT_DIR, folder_name)
            os.makedirs(save_dir, exist_ok=True)
            page_url = f"https://www.jcad.cn/cn/article/{year}/{volume}"
            
            clear_console()
            print(f"===== 开始处理 {year} 年 第 {volume} 卷 =====")
            print(f"📁 保存目录：{save_dir}")
            print(f"🌐 页面URL：{page_url}")
            
            print("\n🔍 解析页面，提取PDF下载链接...")
            pdf_links = extract_pdf_download_urls(page_url, session)
            
            if not pdf_links:
                print(f"⚠️ 未找到任何PDF下载链接，切换下一卷")
            else:
                print(f"\n✅ 共提取到 {len(pdf_links)} 个PDF链接")
                # 分批下载
                batches = [pdf_links[i:i+BATCH_SIZE] for i in range(0, len(pdf_links), BATCH_SIZE)]
                all_success = []
                for batch_idx, batch in enumerate(batches, 1):
                    clear_console()
                    print(f"===== {year} 年 第 {volume} 卷 - 第 {batch_idx} 批（共{len(batches)}批） =====")
                    print(f"📥 本批需下载 {len(batch)} 个文件")
                    batch_success = download_pdf_batch(save_dir, batch, session)
                    all_success.extend(batch_success)
                    print(f"\n✅ 本批完成：{len(batch_success)}/{len(batch)} 个")
                    time.sleep(1)
                
                # 本卷总结
                clear_console()
                print(f"===== {year} 年 第 {volume} 卷 处理完成 =====")
                print(f"📊 总计找到 {len(pdf_links)} 个PDF，成功下载 {len(all_success)} 个")
                if all_success:
                    print("\n📄 已下载文件列表：")
                    for idx, file in enumerate(all_success, 1):
                        file_name = os.path.basename(file)
                        file_size = os.path.getsize(file)/1024
                        print(f"   {idx}. {file_name}（{file_size:.1f}KB）")
            
            # 切换下一卷/年
            print("\n🔄 准备切换到下一卷...")
            if volume <= 1:
                year -= 1
                volume = 12
                if year < MIN_YEAR:
                    print(f"\n🎉 已处理到最小年份 {MIN_YEAR}，任务结束")
                    break
            else:
                volume -= 1
            time.sleep(2)

        except Exception as e:
            print(f"❌ {year} 年 第 {volume} 卷处理失败：{str(e)}")
            import traceback
            traceback.print_exc()
            # 切换下一卷，避免卡死
            if volume <= 1:
                year -= 1
                volume = 12
                if year < MIN_YEAR:
                    break
            else:
                volume -= 1
            time.sleep(2)
            continue

if __name__ == "__main__":
    if not os.path.exists(DOWNLOAD_ROOT_DIR):
        os.makedirs(DOWNLOAD_ROOT_DIR)
    main()

===== 开始处理 2025 年 第 9 卷 =====
📁 保存目录：d:\vscode\Python\Jupyter\jcad\202509
🌐 页面URL：https://www.jcad.cn/cn/article/2025/9

🔍 解析页面，提取PDF下载链接...

📌 共找到 15 个PDF下载按钮

===== 处理第 1 个PDF按钮 =====
✅ 提取到UUID：862e2124-cef0-4b5a-bd45-d63f8d279391
✅ 在第 3 层父元素找到目标标题：【面向高保真材质表示与生成的深度学习技术综述】
📄 最终文件名：【面向高保真材质表示与生成的深度学习技术综述.pdf】

===== 处理第 2 个PDF按钮 =====
✅ 提取到UUID：ab16d2e2-72a9-40d8-8187-a8c6d38ab9c7
✅ 在第 3 层父元素找到目标标题：【基于注意力机制的树木点云补全网络】
📄 最终文件名：【基于注意力机制的树木点云补全网络.pdf】

===== 处理第 3 个PDF按钮 =====
✅ 提取到UUID：b4632fb4-3892-4544-976d-802ac99dce9a
✅ 在第 3 层父元素找到目标标题：【航空母舰多阶段弹药转运序列可视分析】
📄 最终文件名：【航空母舰多阶段弹药转运序列可视分析.pdf】

===== 处理第 4 个PDF按钮 =====
✅ 提取到UUID：98b01113-1e5c-4c6c-86cd-556d433a3fc8
✅ 在第 3 层父元素找到目标标题：【识印鉴章:中国书画印章分析系统】
📄 最终文件名：【识印鉴章_中国书画印章分析系统.pdf】

===== 处理第 5 个PDF按钮 =====
✅ 提取到UUID：5d3cfda2-3879-4f83-9d7f-856e371a95c1
✅ 在第 3 层父元素找到目标标题：【StruMNet:一种融合神经网络和偏微分方程的结构网格生成方法】
📄 最终文件名：【StruMNet_一种融合神经网络和偏微分方程的结构网格生成方法.pdf】

===== 处理第 6 个PDF按钮 =====
✅ 提取到UUID：ebc6c4f2-6894-4ffe-a082-7cde3ec1d0e9
✅ 在第 3 层父元素找到目标标题：【面

Traceback (most recent call last):
  File "d:\NewProgram\python\Lib\site-packages\urllib3\response.py", line 899, in _error_catcher
    yield
  File "d:\NewProgram\python\Lib\site-packages\urllib3\response.py", line 1417, in read_chunked
    chunk = self._handle_chunk(amt)
  File "d:\NewProgram\python\Lib\site-packages\urllib3\response.py", line 1359, in _handle_chunk
    returned_chunk = self._fp._safe_read(self.chunk_left)  # type: ignore[union-attr]
  File "d:\NewProgram\python\Lib\http\client.py", line 648, in _safe_read
    data = self.fp.read(cursize)
  File "d:\NewProgram\python\Lib\socket.py", line 725, in readinto
    return self._sock.recv_into(b)
           ~~~~~~~~~~~~~~~~~~~~^^^
  File "d:\NewProgram\python\Lib\ssl.py", line 1304, in recv_into
    return self.read(nbytes, buffer)
           ~~~~~~~~~^^^^^^^^^^^^^^^^
  File "d:\NewProgram\python\Lib\ssl.py", line 1138, in read
    return self._sslobj.read(len, buffer)
           ~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^
TimeoutError: 

===== 开始处理 2025 年 第 6 卷 =====
📁 保存目录：d:\vscode\Python\Jupyter\jcad\202506
🌐 页面URL：https://www.jcad.cn/cn/article/2025/6

🔍 解析页面，提取PDF下载链接...

📌 共找到 15 个PDF下载按钮

===== 处理第 1 个PDF按钮 =====
✅ 提取到UUID：1e494e96-0313-4746-95e5-77b453af2c17
✅ 在第 3 层父元素找到目标标题：【基于深度学习的大规模室外场景Scan2BIM研究综述】
📄 最终文件名：【基于深度学习的大规模室外场景Scan2BIM研究综述.pdf】

===== 处理第 2 个PDF按钮 =====
✅ 提取到UUID：804c6b02-7b78-4391-98a6-7a6b12e4f0a5
✅ 在第 3 层父元素找到目标标题：【采用动态分组和投票机制的三维模型兴趣点提取方法】
📄 最终文件名：【采用动态分组和投票机制的三维模型兴趣点提取方法.pdf】

===== 处理第 3 个PDF按钮 =====
✅ 提取到UUID：fcc3de38-8234-4b7b-822e-99b82e141f50
✅ 在第 3 层父元素找到目标标题：【RPMNet++:结合Copula去噪模块的双向注意力点云配准网络】
📄 最终文件名：【RPMNet___结合Copula去噪模块的双向注意力点云配准网络.pdf】

===== 处理第 4 个PDF按钮 =====
✅ 提取到UUID：f1a89b76-1179-4726-a19e-8af5effd2073
✅ 在第 3 层父元素找到目标标题：【动态场景下流感知的视觉定位方法】
📄 最终文件名：【动态场景下流感知的视觉定位方法.pdf】

===== 处理第 5 个PDF按钮 =====
✅ 提取到UUID：6df67f21-1567-4157-80a7-b4d559635a52
✅ 在第 3 层父元素找到目标标题：【基于互引导条件耦合的小样本语义分割】
📄 最终文件名：【基于互引导条件耦合的小样本语义分割.pdf】

===== 处理第 6 个PDF按钮 =====
✅ 提取到UUID：b3803a56-1a03-4db0-82f1-c615309